<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [18]:
import os
import pandas as pd, csv, sqlite3
import numpy as np
# names of databases accidents
# names of the tables we collect from the site
names = ['caracteristiques', 'lieux', 'vehicules', 'usagers']

from bs4 import BeautifulSoup as soup
from urllib.request import urlopen as uReq

from random import *
from datetime import datetime


# function to get page soup from html page
def get_page_soup(url):

    # opening connection
    uClient = uReq(url)
    page_html = uClient.read()
    uClient.close()
    #html parser
    return soup(page_html, "html.parser")


# function returning the list of liens to the tables with data
def liens_list():
    #set the url for site gouv.fr
    url = "https://www.data.gouv.fr/fr/datasets/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2019/#_"

    page = get_page_soup(url)
    articles = page.findAll("article")
    liens = [[], [], [], []]
    for a in articles:
        if a.div.h4 != None:
            name = a.div.h4.text.split('-')[0].split('_')[0]
        for i, n in enumerate(names):
            if name == n:
                lien = a.footer.find("div",
                                     {"resource-card-actions btn-toolbar"
                                      }).findAll("a")[1]['href']
                liens[i].append(lien)
    return liens


class AccidentsDataBase:
    def __init__(self, name='accidents.sqlite3'):
        self.name_db = name

        if os.path.isfile(
                self.name_db):  # if sqlite file exist create a connection
            self.connection = self.__connect_db()
            print(self.connection)

        else:  # if file sqlite doesn't exist create file and then create sql

            open(self.name_db, 'x')
            self.connection = self.__connect_db()
            print(self.connection)
            self.__create_db()
            self.prepare_db()

        self.cur = self.connection.cursor()

    def get_tables_names(self):
        self.cur.execute("SELECT name FROM sqlite_master WHERE type='table';")

        #store all names of the tables/chemas in var tables
        tables0 = self.cur.fetchall()

        tables = []
        for tab in tables0:
            tables.append(tab[0])

        return tables
    
    def get_merged_df(self):
        lieux = self.get_df('lieux')
        caracteristiques = self.get_df('caracteristiques')
        caracteristiques_lieux = lieux.join(caracteristiques.set_index('Num_Acc'), on = 'Num_Acc')
        
        
        #usagers_vehichiles = usagers.join(vehicules.set_index('Num_Veh'), on = 'Num_Veh')
        return caracteristiques_lieux

    def get_df(self, table_name):
        return pd.read_sql_query(f"SELECT * FROM {table_name}",
                                 self.connection)

    def __connect_db(self):
        return sqlite3.connect(self.name_db)

    # to create data base on your laptop, call function create_data_base,
    # do not forget to create empty file with name accidents.sqlite3 in the same directory your notebook is.
    def __create_db(self):
        #Import libraries
        import pandas as pd, csv, sqlite3

        # Create sqlite database and cursor
        c = self.connection.cursor()

        #c.execute("""DROP TABLE IF EXISTS caracteristiques, lieux, vehicules, usagers;""")
        c.execute(
            """CREATE TABLE IF NOT EXISTS caracteristiques (Num_Acc INTEGER, jour INTEGER, mois INTEGER, an INTEGER, hrmn TEXT, lum INTEGER, dep TEXT, com TEXT, agg INTEGER, int INTEGER, atm INTEGER, col INTEGER, adr TEXT, gps TEXT, lat TEXT, long TEXT)"""
        )
        c.execute(
            """CREATE TABLE IF NOT EXISTS lieux (Num_Acc INTEGER, catr INTEGER, voie REAL, v1 REAL, v2 TEXT, circ REAL, nbv REAL, pr REAL, pr1 REAL, vosp REAL, prof REAL, plan REAL, lartpc REAL, larrout REAL, surf REAL, infra REAL, situ REAL, env1 REAL, vma INTEGER)"""
        )
        c.execute(
            """CREATE TABLE IF NOT EXISTS vehicules (Num_Acc INTEGER, id_vehicule TEXT, senc REAL, catv INTEGER, occutc INTEGER, obs REAL, obsm REAL, choc REAL, manv REAL, num_veh TEXT, motor INTEGER)"""
        )
        c.execute(
            """CREATE TABLE IF NOT EXISTS usagers (Num_Acc INTEGER, id_vehicule TEXT, place REAL, catu INTEGER, grav INTEGER, sexe INTEGER, trajet REAL, secu REAL, secu1 REAL, secu2 REAL, secu3 REAL, locp REAL, actp REAL, etatp REAL, an_nais REAL, num_veh TEXT)"""
        )
        self.connection.commit()

        for i, lien in enumerate(liens_list()):  #names 0,1,2,3
            for j, l in enumerate(lien):  #years 0-15
                print("fichier {} - {}".format(names[i], 2019 - j))
                #print(l)
                try:
                    df = pd.read_csv(
                        l, sep=';', low_memory=False
                    )  #, encoding = 'ISO-8859-1', error_bad_lines=False, quotechar='"'

                    if (len(df.columns) < 2):
                        df = pd.read_csv(l, sep=',', low_memory=False)
                except:
                    try:
                        df = pd.read_csv(l,
                                         sep=',',
                                         encoding='ISO-8859-1',
                                         engine='python')
                        print('exeption1')
                    except:
                        df = pd.read_csv(l,
                                         sep='\t',
                                         encoding='ISO-8859-1',
                                         engine='python')
                        print('exeption2')

                df.to_sql(names[i],
                          self.connection,
                          if_exists='append',
                          index=False)

        print(f'Data base {self.name_db} was created')


    def prepare_db(self):
        #do if your data base is not clean yet
        try:
            self.clean_caracteristiques()
        except:
            print('Error to clean caracteristiques table')
            
        try:
            self.clean_lieux()
        except:
            print('Error to clean lieux table')
            
        ### do clean of the rest tables here
        
    
    def clean_caracteristiques(self):
        caracteristiques = self.get_df('caracteristiques')
        
        drop_caracteristiques = ['gps', 'adr', 'com']

        new_values = \
    ['01', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19',
        '02', '21', '22', '23', '24', '25', '26', '27', '28', '29',
        '2A', '2B',
        '03', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39',
        '04', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49',
        '05', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59',
        '06', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69',
        '07', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79',
        '08', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89',
        '09', '90', '91', '92', '93', '94', '95',
        'QP', 'MQ', 'GF', 'RE', 'YT']

        old_values = \
    ['10', '100', '110', '120', '130', '140', '150', '160', '170', '180', '190',
        '20', '210', '220', '230', '240', '250', '260', '270', '280', '290',
        '201', '202',
        '30', '300', '310', '320', '330', '340', '350', '360', '370', '380', '390',
        '40', '400', '410', '420', '430', '440', '450', '460', '470', '480', '490',
        '50', '500', '510', '520', '530', '540', '550', '560', '570', '580', '590',
        '60', '600', '610', '620', '630', '640', '650', '660', '670', '680', '690',
        '70', '700', '710', '720', '730', '740', '750', '760', '770', '780', '790',
        '80', '800', '810', '820', '830', '840', '850', '860', '870', '880', '890',
        '90', '900', '910', '920', '930', '940', '950',
        '971', '972', '973', '974', '976']

        # drop columns
        caracteristiques = caracteristiques.drop(drop_caracteristiques, axis=1)
        # clean dep
        caracteristiques.dep = caracteristiques.dep.replace(old_values, new_values)
        # clean an
        caracteristiques.loc[(caracteristiques.an < 1000),
                             'an'] = caracteristiques.an + 2000
        caracteristiques.an = caracteristiques.an.astype('int')

        # hrmn
        for ind in caracteristiques[(caracteristiques.hrmn.str.len() < 3)].index:
            #print(len(caracteristiques[(caracteristiques.hrmn.str.len() < 3)].index))
            #print(ind)
            if caracteristiques.loc[ind, 'lum'] == 1:
                hm = str(choice([9, 10, 11, 12, 13, 14, 15, 16, 17
                                 ])).zfill(2) + ':' + str(randint(0, 59)).zfill(2)
            if caracteristiques.loc[ind, 'lum'] == 2:
                hm = str(choice([6, 7, 19, 20])).zfill(2) + ':' + str(
                    randint(0, 59)).zfill(2)
            if caracteristiques.loc[ind, 'lum'] in [3, 4, 5]:
                hm = str(choice([23, 0, 1, 2, 3, 4])).zfill(2) + ':' + str(
                    randint(0, 59)).zfill(2)

            caracteristiques.loc[ind, 'hrmn'] = hm


        # separate numbers with : for time format
        caracteristiques.hrmn = [
            hm[:-2] + ':' + hm[-2:] if ':' not in hm else hm
            for hm in caracteristiques.hrmn
        ]

        ## change format str to time object

        caracteristiques.hrmn = [
            datetime.strptime(hm, '%H:%M').time()
            if type(hm) != type(datetime.now().time()) else hm
            for hm in caracteristiques.hrmn
        ]

        # weekday new columns
        caracteristiques['weekday'] = [datetime(an, mois, jour).weekday() for an,mois,jour in zip(caracteristiques.an, caracteristiques.mois, caracteristiques.jour)]

        #lat
        caracteristiques['lat'] = caracteristiques['lat'].str.replace(',','.')
        caracteristiques.lat = caracteristiques.lat.astype(float)
        caracteristiques.loc[(caracteristiques.lat > 10000),'lat'] = caracteristiques.lat/100000

        #long   
        caracteristiques['long'] = caracteristiques['long'].str.replace(',','.')
        caracteristiques['long'] = [np.nan if l == '-' else l for l in caracteristiques['long']]
        caracteristiques['long'] = caracteristiques['long'].astype(float)
        caracteristiques.loc[(caracteristiques['an'] != 2019),'long'] = caracteristiques['long']/100000

        #atm
        caracteristiques.atm = caracteristiques.atm.replace([np.nan,-1], 1)
        
        self.cur.execute("""DROP TABLE IF EXISTS caracteristiques;""")    
        caracteristiques.to_sql('caracteristiques',
                          self.connection, if_exists = 'replace',
                          index=False)
        
        
    def clean_lieux(self):
        
        lieux = self.get_df('lieux')
        
        # do cleaning here
        lieux = lieux.drop(['v1','v2','vma', 'env1'], axis = 1)
        
        lieux.catr = lieux.catr.replace(np.nan, 4)
        
        lieux.circ = lieux.circ.replace([np.nan, -1], 0)
        
        lieux.nbv = lieux.nbv.replace([-1.,np.nan], 0.)
        
        ## Assumtion 1
        ## Important assumtion cut all accidents where number of roads >10 or negative! Delete around 800 cases
        lieux = lieux[(lieux.nbv <= 10) | (lieux.nbv<0)]
        
        lieux.vosp = lieux.vosp.replace([-1., np.nan], 0.)
        lieux.vosp = lieux.vosp.astype(int)
        
        lieux.prof = lieux.prof.replace([np.nan,-1.], 1.)
        
        lieux.pr = lieux.pr.replace([np.nan,'(1)', ''], -1.0)
        lieux.pr1 = lieux.pr1.replace([np.nan,'(1)', ''], -1.0)
        
        lieux.plan= lieux.plan.replace([-1.,np.nan],0.)
        
        lieux.surf = lieux.surf.replace([np.nan,-1.,0.])

        lieux.infra = lieux.infra.replace([np.nan, -1.], 0.)                             
        lieux.situ = lieux.situ.replace([np.nan, -1], 1)       
                                        
        ## Assumtion 2
        #assumtion
        lieux.lartpc = lieux.lartpc.replace(np.nan, 0.)
        lieux.larrout = lieux.larrout.replace(np.nan, 0.)
        
        # drop old table
        self.cur.execute("""DROP TABLE IF EXISTS caracteristiques;""")    
        
        #put clean table to the db
        lieux.to_sql('lieux',
                          self.connection, if_exists = 'replace',
                          index=False)
        
        
        
      

In [19]:
db = AccidentsDataBase()

In [20]:
db.get_tables_names()

['vehicules', 'usagers', 'lieux', 'caracteristiques']

In [4]:
db.get_merged_df()

,Num_Acc,jour,mois,an,hrmn,lum,dep,agg,int,atm,...,pr,pr1,vosp,prof,plan,lartpc,larrout,surf,infra,situ
0,201900000001,30,11,2019,01:30:00,4,93,1,1,1.0,...,6.0,900.0,0.0,1.0,2.0,0.0,0.0,1.0,2.0,1.0
1,201900000002,30,11,2019,02:50:00,3,93,1,1,1.0,...,3.0,845.0,0.0,4.0,2.0,0.0,0.0,1.0,0.0,1.0
2,201900000003,28,11,2019,15:15:00,1,92,1,1,1.0,...,10.0,500.0,0.0,1.0,3.0,0.0,0.0,1.0,0.0,1.0
3,201900000004,30,11,2019,20:20:00,5,94,1,1,1.0,...,2.0,299.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0
4,201900000005,30,11,2019,04:00:00,3,94,1,1,1.0,...,41.0,0.0,0.0,1.0,3.0,0.0,0.0,1.0,2.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1017304,200500087950,21,12,2005,20:35:00,5,RE,2,2,1.0,...,-1.0,-1.0,0.0,1.0,1.0,0.0,80.0,1.0,0.0,1.0
1017305,200500087951,23,12,2005,10:10:00,1,RE,2,1,1.0,...,-1.0,-1.0,0.0,1.0,2.0,0.0,70.0,1.0,0.0,1.0
1017306,200500087952,26,12,2005,17:15:00,1,RE,2,2,1.0,...,-1.0,-1.0,0.0,1.0,1.0,0.0,75.0,1.0,0.0,1.0
1017307,200500087953,27,12,2005,15:00:00,1,RE,2,3,2.0,...,-1.0,-1.0,0.0,1.0,1.0,0.0,150.0,1.0,0.0,1.0


In [73]:
df1 = db.get_df('caracteristiques')

In [7]:
from functions import *

In [75]:
df_info(df1)


    Number of lines : 1017309
    Number of columns: 16
    


,name,type,n_unique,unique,null%
13,gps,object,11,"[None, M, A, G, R, Y, S, T, C, P, 0]",52.81
14,lat,object,231257,"[48,8962100, 48,9307000, 48,9358718, 48,817329...",47.88
15,long,object,264939,"[2,4701200, 2,3688000, 2,3191744, 2,4281502, 2...",47.88
12,adr,object,437236,"[AUTOROUTE A3, AUTOROUTE A1, AUTOROUTE A86, A4...",13.99
10,atm,float64,11,"[-1.00e+00, 1.56e+00, 9.00e+00]",0.01
0,Num_Acc,int64,1017309,"[2.01e+11, 2.01e+11, 2.02e+11]",0.00
1,jour,int64,31,"[1.00e+00, 1.56e+01, 3.10e+01]",0.00
2,mois,int64,12,"[1.00e+00, 6.68e+00, 1.20e+01]",0.00
3,an,int64,15,"[5.00e+00, 1.27e+02, 2.02e+03]",0.00
4,hrmn,object,2825,"[01:30, 02:50, 15:15, 20:20, 04:00, 14:03, 07:...",0.00


In [11]:
caracteristiques = pd.read_csv('caracteristique_2005_2019.csv', low_memory=True)

db.cur.execute("""DROP TABLE IF EXISTS caracteristiques;""")

caracteristiques.to_sql('caracteristiques',
                          db.connection, if_exists = 'replace',
                          index=False)

In [78]:
df2 = db.get_df('caracteristiques')
df_info(df2)


    Number of lines : 1017309
    Number of columns: 14
    


,name,type,n_unique,unique,null%
12,long,float64,254506,"[-1.78e+02, 2.38e+01, 6.13e+06]",48.19
11,lat,float64,220405,"[-6.14e+01, 3.41e+01, 6.36e+01]",47.88
0,Num_Acc,int64,1017309,"[2.01e+11, 2.01e+11, 2.02e+11]",0.00
1,jour,int64,31,"[1.00e+00, 1.56e+01, 3.10e+01]",0.00
2,mois,int64,12,"[1.00e+00, 6.68e+00, 1.20e+01]",0.00
3,an,int64,15,"[2.00e+03, 2.01e+03, 2.02e+03]",0.00
4,hrmn,object,1440,"[01:30:00, 02:50:00, 15:15:00, 20:20:00, 04:00...",0.00
5,lum,int64,5,"[1.00e+00, 1.91e+00, 5.00e+00]",0.00
6,dep,object,116,"[93, 92, 94, 87, 69, 38, 34, 13, 988, YT, RE, ...",0.00
7,agg,int64,2,"[1.00e+00, 1.68e+00, 2.00e+00]",0.00


In [5]:
lieux = pd.read_csv('lieux_2005_2019.csv', low_memory=True)

db.cur.execute("""DROP TABLE IF EXISTS lieux;""")

lieux.to_sql('lieux',
                          db.connection, if_exists = 'replace',
                          index=False)

In [8]:
df3 = db.get_df('lieux')
df_info(df3)


    Number of lines : 1016551
    Number of columns: 15
    


,name,type,n_unique,unique,null%
2,voie,object,14860,"[3.0, 1.0, 86.0, 4.0, FOCH (RUE DU MARECHAL), ...",9.03
0,Num_Acc,int64,1016551,"[2.01e+11, 2.01e+11, 2.02e+11]",0.00
1,catr,float64,8,"[1.00e+00, 3.40e+00, 9.00e+00]",0.00
3,circ,float64,5,"[0.00e+00, 1.86e+00, 4.00e+00]",0.00
4,nbv,float64,11,"[0.00e+00, 2.06e+00, 1.00e+01]",0.00
5,pr,float64,824,"[-1.00e+00, 4.70e+01, 9.93e+03]",0.00
6,pr1,float64,2267,"[-1.00e+00, 1.42e+02, 9.54e+03]",0.00
7,vosp,int64,4,"[0.00e+00, 1.33e-01, 3.00e+00]",0.00
8,prof,float64,5,"[0.00e+00, 1.15e+00, 4.00e+00]",0.00
9,plan,float64,5,"[0.00e+00, 1.20e+00, 4.00e+00]",0.00


In [10]:
db.get_tables_names()

['vehicules', 'usagers', 'caracteristiques', 'lieux']

In [12]:
lieux.head(2)

,Num_Acc,catr,voie,circ,nbv,pr,pr1,vosp,prof,plan,lartpc,larrout,surf,infra,situ
0,201900000001,1.0,3.0,3.0,10.0,6.0,900.0,0,1.0,2.0,0.0,0.0,1.0,2.0,1.0
1,201900000002,1.0,1.0,1.0,2.0,3.0,845.0,0,4.0,2.0,0.0,0.0,1.0,0.0,1.0


In [13]:
caracteristiques.head(2)

,Num_Acc,jour,mois,an,hrmn,lum,dep,agg,int,atm,col,lat,long,weekday
0,201900000001,30,11,2019,01:30:00,4,93,1,1,1.0,2.0,48.89621,2.47012,5
1,201900000002,30,11,2019,02:50:00,3,93,1,1,1.0,6.0,48.93070,2.36880,5


In [14]:
merged = caracteristiques.join(lieux.set_index('Num_Acc'), on = 'Num_Acc')

In [15]:
merged.head(2)

,Num_Acc,jour,mois,an,hrmn,lum,dep,agg,int,atm,col,lat,long,weekday,catr,voie,circ,nbv,pr,pr1,vosp,prof,plan,lartpc,larrout,surf,infra,situ
0,201900000001,30,11,2019,01:30:00,4,93,1,1,1.0,2.0,48.89621,2.47012,5,1.0,3.0,3.0,10.0,6.0,900.0,0.0,1.0,2.0,0.0,0.0,1.0,2.0,1.0
1,201900000002,30,11,2019,02:50:00,3,93,1,1,1.0,6.0,48.93070,2.36880,5,1.0,1.0,1.0,2.0,3.0,845.0,0.0,4.0,2.0,0.0,0.0,1.0,0.0,1.0


In [5]:
vehicules  = db.get_df('vehicules')

In [7]:
vehicules.head(2)

,Num_Acc,id_vehicule,senc,catv,occutc,obs,obsm,choc,manv,num_veh,motor
0,201900000001,138 306 524,2.0,7,NaN,0.0,2.0,5.0,23.0,B01,1.0
1,201900000001,138 306 525,2.0,17,NaN,1.0,0.0,3.0,11.0,A01,1.0


In [8]:
vehicules.num_veh.value_counts()

A01     1003212
B01      493306
B02      128574
C01       55768
Z01       19572
         ...   
MA01          1
W49           1
G06           1
VB01          1
G04           1
Name: num_veh, Length: 155, dtype: int64

In [21]:

db_test = AccidentsDataBase('test_db.sqlite3')


In [ ]:
db_test.prepare_db()